In [ ]:
# install pip dependencies
! pip install -q kaggle
! pip install pandas
! pip install torch datasets
! pip install ipdb

^C


The process cannot access the file because it is being used by another process.
The process cannot access the file because it is being used by another process.
The process cannot access the file because it is being used by another process.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# kaggle authentication
# upload kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# import libraries
import kagglehub
import os
import pandas as pd
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import gc
import numpy as np
from tqdm import tqdm
import io
import tarfile

In [ ]:
# download data to paraquet
download_path = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")
raw_csv_files = glob.glob(os.path.join(download_path, "*.csv"))
print(raw_csv_files)

['/kaggle/input/ids-intrusion-csv/02-28-2018.csv', '/kaggle/input/ids-intrusion-csv/03-01-2018.csv', '/kaggle/input/ids-intrusion-csv/02-16-2018.csv', '/kaggle/input/ids-intrusion-csv/02-15-2018.csv', '/kaggle/input/ids-intrusion-csv/02-21-2018.csv', '/kaggle/input/ids-intrusion-csv/03-02-2018.csv', '/kaggle/input/ids-intrusion-csv/02-22-2018.csv', '/kaggle/input/ids-intrusion-csv/02-20-2018.csv', '/kaggle/input/ids-intrusion-csv/02-14-2018.csv', '/kaggle/input/ids-intrusion-csv/02-23-2018.csv']


In [ ]:
# split csvs into managable chunks

CSV_FILES = []

num_raw_files = len(raw_csv_files)
for i, file in enumerate(raw_csv_files):
  print(f"splitting: {i+1}/{num_raw_files}")
  filename = os.path.splitext(os.path.basename(file))[0]
  max_bytes = 500 * 1024 * 1024

  part_idx = 1
  current_size = 0

  with open(file, 'r') as infile:
    header = infile.readline()
    lines_buffer = []
    for line in infile:
      lines_buffer.append(line)
      current_size += len(line.encode('utf-8'))  # size in bytes

      # If current chunk exceeds the limit, write to file
      if current_size >= max_bytes:
        out_path = os.path.join(f"/content/{filename}_part{part_idx}.csv")
        with open(out_path, 'w') as outfile:
          outfile.write(header)
          outfile.writelines(lines_buffer)
        CSV_FILES.append(out_path)
        part_idx += 1
        lines_buffer = []
        current_size = 0

    # Write remaining lines
    if lines_buffer:
      out_path = os.path.join(f"/content/{filename}_part{part_idx}.csv")
      with open(out_path, 'w') as outfile:
        outfile.write(header)
        outfile.writelines(lines_buffer)
      CSV_FILES.append(out_path)

splitting: 1/10
splitting: 2/10
splitting: 3/10
splitting: 4/10
splitting: 5/10
splitting: 6/10
splitting: 7/10
splitting: 8/10
splitting: 9/10
splitting: 10/10


In [ ]:
test_df = pd.read_csv(CSV_FILES[1])
NUM_COLUMNS = len(test_df.columns)
print(test_df.columns)
del test_df
gc.collect()

/tmp/ipython-input-6-1566101961.py:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(CSV_FILES[1])


Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

30

In [ ]:
# canon columns
CANON_COLUMN_INDEX = ['Fwd IAT Tot', 'Fwd Pkt Len Min', 'Down/Up Ratio', 'Dst Port', 'Fwd IAT Std', 'Fwd Header Len', 'Fwd IAT Min', 'Flow IAT Std', 'Active Std', 'Bwd IAT Max', 'Fwd Pkt Len Mean', 'Pkt Size Avg', 'PSH Flag Cnt', 'Flow IAT Mean', 'Fwd Act Data Pkts', 'Bwd Pkt Len Max', 'Flow IAT Max', 'ACK Flag Cnt', 'Bwd IAT Tot', 'Flow IAT Min', 'Bwd Pkts/b Avg', 'Fwd IAT Max', 'SYN Flag Cnt', 'Bwd Header Len', 'Fwd Seg Size Avg', 'Bwd Byts/b Avg', 'Subflow Bwd Byts', 'Pkt Len Max', 'Bwd Pkts/s', 'Fwd IAT Mean', 'Pkt Len Var', 'Fwd Pkt Len Std', 'Protocol', 'Init Bwd Win Byts', 'Active Min', 'Src Port', 'RST Flag Cnt', 'Subflow Fwd Byts', 'Init Fwd Win Byts', 'Bwd Pkt Len Std', 'Fwd PSH Flags', 'Fwd Pkts/s', 'Bwd Blk Rate Avg', 'Flow Byts/s', 'CWE Flag Count', 'Pkt Len Std', 'Active Max', 'Fwd Byts/b Avg', 'Fwd Blk Rate Avg', 'URG Flag Cnt', 'Timestamp', 'Fwd Pkts/b Avg', 'Idle Mean', 'Idle Std', 'Fwd Pkt Len Max', 'Pkt Len Min', 'Flow Duration', 'Fwd Seg Size Min', 'Bwd IAT Min', 'TotLen Fwd Pkts', 'Flow Pkts/s', 'Active Mean', 'ECE Flag Cnt', 'Idle Min', 'Subflow Bwd Pkts', 'Bwd Pkt Len Mean', 'Pkt Len Mean', 'Tot Fwd Pkts', 'Bwd IAT Std', 'Bwd Seg Size Avg', 'Bwd URG Flags', 'Bwd Pkt Len Min', 'Tot Bwd Pkts', 'Subflow Fwd Pkts', 'Bwd IAT Mean', 'FIN Flag Cnt', 'Bwd PSH Flags', 'TotLen Bwd Pkts', 'Fwd URG Flags', 'Idle Max']
CANON_COLUMN_INDEX.sort()
CANON_COLUMN_INDEX.append('Label')
print(CANON_COLUMN_INDEX)
TRAINING_UNWANTED_COLUMNS = ['Timestamp', 'Flow ID', 'Dst IP', "Src IP"]

['ACK Flag Cnt', 'Active Max', 'Active Mean', 'Active Min', 'Active Std', 'Bwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Header Len', 'Bwd IAT Max', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd IAT Std', 'Bwd IAT Tot', 'Bwd PSH Flags', 'Bwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Min', 'Bwd Pkt Len Std', 'Bwd Pkts/b Avg', 'Bwd Pkts/s', 'Bwd Seg Size Avg', 'Bwd URG Flags', 'CWE Flag Count', 'Down/Up Ratio', 'Dst Port', 'ECE Flag Cnt', 'FIN Flag Cnt', 'Flow Byts/s', 'Flow Duration', 'Flow IAT Max', 'Flow IAT Mean', 'Flow IAT Min', 'Flow IAT Std', 'Flow Pkts/s', 'Fwd Act Data Pkts', 'Fwd Blk Rate Avg', 'Fwd Byts/b Avg', 'Fwd Header Len', 'Fwd IAT Max', 'Fwd IAT Mean', 'Fwd IAT Min', 'Fwd IAT Std', 'Fwd IAT Tot', 'Fwd PSH Flags', 'Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Min', 'Fwd Pkt Len Std', 'Fwd Pkts/b Avg', 'Fwd Pkts/s', 'Fwd Seg Size Avg', 'Fwd Seg Size Min', 'Fwd URG Flags', 'Idle Max', 'Idle Mean', 'Idle Min', 'Idle Std', 'Init Bwd Win Byts', 'Init Fwd Win Byts', 'PSH Flag Cnt

In [ ]:
# get normalization stats
UNWANTED_COLUMNS_STATS = ['Timestamp','Label', 'Flow ID', 'Dst IP', "Src IP"]

def calcStatistics(files):
  n_total = 0
  mean_total = pd.Series(0.0)
  M2 = pd.Series(0.0)

  file_idx = 0
  num_files = len(files)
  for file in files:
    print(f"calculating: {file_idx+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')

        if col not in mean_total:
          mean_total[col] = 0.0
          M2[col] = 0.0

      else:
        df = df.drop(columns=[col])

    n = len(df)
    mean = df.mean()
    var = df.var(ddof=0)

    # Welford’s algorithm for combining means and variances
    delta = mean - mean_total
    new_n = n_total + n

    mean_total = (n_total * mean_total + n * mean) / new_n
    M2 += var * n + (delta**2) * n_total * n / new_n

    n_total = new_n

    file_idx += 1
    del df
    gc.collect()

  std = (M2 / n_total) ** 0.5
  length = n_total
  mean = mean_total
  return (length, mean, std)

LENGTH, MEAN, STD = calcStatistics(CSV_FILES)

import json
j = {'length': LENGTH, 'mean': MEAN.to_dict(), 'std': STD.to_dict(), 'non-stat-columns': UNWANTED_COLUMNS_STATS}
with open('info.json', 'w') as f:
  json.dump(j,f)

calculating: 1/17


/tmp/ipython-input-9-1231377406.py:13: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


calculating: 2/17


/tmp/ipython-input-9-1231377406.py:13: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


calculating: 3/17


/tmp/ipython-input-9-1231377406.py:13: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


calculating: 4/17
calculating: 5/17
calculating: 6/17
calculating: 7/17
calculating: 8/17
calculating: 9/17
calculating: 10/17
calculating: 11/17
calculating: 12/17
calculating: 13/17
calculating: 14/17
calculating: 15/17
calculating: 16/17
calculating: 17/17
16233002 0                            NaN
ACK Flag Cnt            0.331602
Active Max         262392.669868
Active Mean        172878.053685
Active Min         115466.001905
                       ...      
Tot Bwd Pkts            6.312703
Tot Fwd Pkts           23.533122
TotLen Bwd Pkts      4730.926465
TotLen Fwd Pkts       973.035995
URG Flag Cnt            0.041713
Length: 80, dtype: float64 0                           NaN
ACK Flag Cnt       4.704343e-01
Active Max         3.310166e+06
Active Mean        2.498837e+06
Active Min         2.109640e+06
                       ...     
Tot Bwd Pkts       1.639441e+02
Tot Fwd Pkts       1.520272e+03
TotLen Bwd Pkts    2.344114e+05
TotLen Fwd Pkts    6.214612e+04
URG Flag Cnt       2.

In [ ]:
# normalize dataset and convert to parquet
! mkdir "normalized/"

UNWANTED_COLUMNS_NORM = ['Flow ID', 'Dst IP', "Src IP"]

def normalize(files, mean, std):
  pq_files = []

  num_files = len(files)
  for i,file in enumerate(CSV_FILES):
    print(f"normalizing: {i+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    filename = os.path.splitext(os.path.basename(file))[0]
    name = f'/content/normalized/normalized_{filename}.parquet'
    pq_files.append(name)

    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
          df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')
          df[col] = (df[col] - mean[col]) / std[col]
      else:
        if col in UNWANTED_COLUMNS_NORM:
          df = df.drop(columns=[col])

    if 'Src Port' not in df.columns:
      df['Src Port'] = np.nan

    df.to_parquet(name)

    del df
    gc.collect()
  return pq_files

PARQUET_FILES = normalize(CSV_FILES, MEAN, STD)
print(PARQUET_FILES)

normalizing: 1/17


/tmp/ipython-input-10-3407048668.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 2/17


/tmp/ipython-input-10-3407048668.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 3/17


/tmp/ipython-input-10-3407048668.py:12: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


normalizing: 4/17
normalizing: 5/17
normalizing: 6/17
normalizing: 7/17
normalizing: 8/17
normalizing: 9/17
normalizing: 10/17
normalizing: 11/17
normalizing: 12/17
normalizing: 13/17
normalizing: 14/17
normalizing: 15/17
normalizing: 16/17
normalizing: 17/17
['/content/normalized/normalized_02-28-2018_part1.parquet', '/content/normalized/normalized_03-01-2018_part1.parquet', '/content/normalized/normalized_02-16-2018_part1.parquet', '/content/normalized/normalized_02-15-2018_part1.parquet', '/content/normalized/normalized_02-21-2018_part1.parquet', '/content/normalized/normalized_03-02-2018_part1.parquet', '/content/normalized/normalized_02-22-2018_part1.parquet', '/content/normalized/normalized_02-20-2018_part1.parquet', '/content/normalized/normalized_02-20-2018_part2.parquet', '/content/normalized/normalized_02-20-2018_part3.parquet', '/content/normalized/normalized_02-20-2018_part4.parquet', '/content/normalized/normalized_02-20-2018_part5.parquet', '/content/normalized/normalized

In [ ]:
# double check columns
base_df = pd.read_parquet(PARQUET_FILES[0])
base = set(list(base_df.columns))
del base_df
gc.collect()

for f in PARQUET_FILES:
  df = pd.read_parquet(f)
  cols = set(list(df.columns))
  if base != cols:
    print(f'in base: {base - cols}, in cols: {cols - base}')
  del df
  gc.collect()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
! tar -czvf normalized-ids2018-parquet.tar.gz normalized/*.parquet info.json
# ! cp normalized-ids2018.tar.gz drive/MyDrive/

normalized/normalized_02-14-2018_part1.parquet
normalized/normalized_02-15-2018_part1.parquet
normalized/normalized_02-16-2018_part1.parquet
normalized/normalized_02-20-2018_part1.parquet
normalized/normalized_02-20-2018_part2.parquet
normalized/normalized_02-20-2018_part3.parquet
normalized/normalized_02-20-2018_part4.parquet
normalized/normalized_02-20-2018_part5.parquet
normalized/normalized_02-20-2018_part6.parquet
normalized/normalized_02-20-2018_part7.parquet
normalized/normalized_02-20-2018_part8.parquet
normalized/normalized_02-21-2018_part1.parquet
normalized/normalized_02-22-2018_part1.parquet
normalized/normalized_02-23-2018_part1.parquet
normalized/normalized_02-28-2018_part1.parquet
normalized/normalized_03-01-2018_part1.parquet
normalized/normalized_03-02-2018_part1.parquet
info.json


In [ ]:
! rm ./*.csv